### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check , same type, same cam , timestamp in the lifespan +- grace windows (padding periods) x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [6]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-12"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())
df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_SNAPSHOT]

# drop the one that lifetime is less than 4 seconds
# df_vehicle = df_vehicle.loc[(df_vehicle["lifetime"] > 4),]

# create a timestamp_unix at the end of lifespan
# df_vehicle["timestamp_unix_end"] = df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"].astype(np.int64))
print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())

                 camera       timestamp_precise    xmid   ymid  lifetime plate_number vehicle_type  timestamp_unix  timestamp_unix_end  timestamp_unix  timestamp_unix_end
2315  mbk-14-12-vehicle 2024-04-28 02:27:41.480  1706.0  194.0    14.236       327722       pickup      1714271261          1714271275      1714271261          1714271275
2242  mbk-14-12-vehicle 2024-04-28 03:08:09.313  1564.5  181.5    18.355      4กบ5887       pickup      1714273689          1714273707      1714273689          1714273707
2241  mbk-14-12-vehicle 2024-04-28 03:08:27.068  1397.0  140.5    49.866      6กถ9907       pickup      1714273707          1714273756      1714273707          1714273756
2215  mbk-14-12-vehicle 2024-04-28 03:18:58.899  1551.5  172.0    46.827          อก5       pickup      1714274338          1714274384      1714274338          1714274384
2204  mbk-14-12-vehicle 2024-04-28 03:21:31.980  1401.5  174.0     2.639       ทห1197       pickup      1714274491          1714274493      17142

## Cross vs Vehicle

In [7]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

## Reverse vs Vehicle

In [8]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

## TOGETHER


In [9]:
df_vehicle["lifetime"]

2315      14.236
2242      18.355
2241      49.866
2215      46.827
2204       2.639
          ...   
233      220.742
234        7.598
211       86.856
152       26.473
70      2885.068
Name: lifetime, Length: 113, dtype: float64

In [10]:
df_vehicle = match_reverse_to_vehicle(df_reverse, df_vehicle)
df_vehicle = match_cross_to_vehicle(
    df_cross,
    df_vehicle,
)


df_vehicle = df_vehicle[
    (df_vehicle["timestamp_unix"] >= 1714284000) & (df_vehicle["timestamp_unix_end"] <= 1714288020)
]
df_vehicle = df_vehicle[
    [
        *conf.VEHICLE_INTEREST_SNAPSHOT,
        "cross_count",
        "reverse_count",
        "original_video_name",
        # "frame_no",
    ]
]

df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)
print(df_vehicle.to_string())

                 camera       timestamp_precise    xmid   ymid  lifetime plate_number vehicle_type  timestamp_unix  timestamp_unix_end  cross_count  reverse_count               original_video_name
1845  mbk-14-12-vehicle 2024-04-28 06:11:19.503  1713.0  194.0   208.942       ฮฮ5951       pickup      1714284679          1714284887           11              2  mbk-14-12-vehicle_1714284601.mp4
1829  mbk-14-12-vehicle 2024-04-28 06:19:21.892  1749.5  197.5    88.375       ฮค9016       pickup      1714285161          1714285249            3              0  mbk-14-12-vehicle_1714284901.mp4
1812  mbk-14-12-vehicle 2024-04-28 06:27:25.085  1573.5  177.5     6.358       นษ9532          suv      1714285645          1714285651            0              0  mbk-14-12-vehicle_1714285501.mp4
1795  mbk-14-12-vehicle 2024-04-28 06:29:54.323  1707.0  192.0   116.171          NaN          NaN      1714285794          1714285910           19              0  mbk-14-12-vehicle_1714285501.mp4
1794  mbk-14-12

## TODO 
- check cross walk angle?
- allow customer able to be classified to the next car (not found in the current time stamp)?
- check displacement?